### 🔑 Part 4 연습문제 정답

##### ✏️ 연습문제 1: 최적의 `n_components` 찾기

PCA를 사용할 때, 정보 손실을 최소화하면서 차원을 얼마나 줄일지 결정하는 것이 중요합니다. 

`PCA`의 `explained_variance_ratio_` 속성을 이용하여 **분산의 95%를 설명하기 위해 몇 개의 주성분이 필요한지** 계산하고, 결과를 시각화해보세요.

In [ ]:
# 연습문제 1 코드
# StandardScaler로 전체 샘플 데이터(X_sample) 스케일링
scaler = StandardScaler()
X_sample_scaled = scaler.fit_transform(X_sample)

# PCA 객체 생성 (모든 주성분을 확인하기 위해 n_components를 설정하지 않음)
pca = PCA()
pca.fit(X_sample_scaled)

# 누적 설명 분산 계산
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)

# 95% 분산을 설명하는 지점 찾기
# 여기에 코드를 작성하여 n_components_95를 구하세요.
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
print(f"95%의 분산을 설명하는 주성분 개수: {n_components_95}")


# 누적 설명 분산 시각화
fig = px.area(
    x=range(1, cumulative_variance.shape[0] + 1),
    y=cumulative_variance,
    labels={"x": "주성분 개수", "y": "누적 설명 분산 비율"},
    title="PCA 누적 설명 분산"
)
fig.add_shape(type="line", x0=0, y0=0.95, x1=n_components_95, y1=0.95, line=dict(color="Red", dash="dash"))
fig.add_shape(type="line", x0=n_components_95, y0=0, x1=n_components_95, y1=0.95, line=dict(color="Red", dash="dash"))
fig.show()

##### ✏️ 연습문제 2: `GridSearchCV`로 최적의 SVM 파라미터 찾기

`Pipeline`과 `GridSearchCV`를 함께 사용하면 전처리 파라미터와 모델 하이퍼파라미터를 동시에 최적화할 수 있습니다.

 위에서 구한 `n_components_95`를 PCA에 적용하고, SVM의 `C`와 `gamma`에 대한 최적값을 찾아보세요.

In [ ]:
from sklearn.model_selection import GridSearchCV

# 연습문제 2 코드
# 1. 파이프라인 정의
# n_components는 연습문제 1에서 구한 값으로 설정
pipe_for_grid = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=n_components_95)), # n_components_95 값 사용
    ('svm', SVC(kernel='rbf', random_state=42))
])

# 2. 탐색할 하이퍼파라미터 그리드 정의
# 파이프라인의 각 단계 이름과 파라미터 이름을 '__'로 연결
param_grid = {
    'svm__C': [0.1, 1, 10],
    'svm__gamma': ['scale', 'auto', 0.1, 1]
}

# 3. GridSearchCV 객체 생성 및 학습
# cv=3 (3-fold cross-validation)
# n_jobs=-1 (사용 가능한 모든 CPU 코어 사용)
grid_search = GridSearchCV(pipe_for_grid, param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# 4. 최적 파라미터 및 성능 출력
print("최적 하이퍼파라미터:", grid_search.best_params_)
print("최고 교차 검증 점수:", grid_search.best_score_)
print("테스트 세트 점수:", grid_search.score(X_test, y_test))

##### ✏️ 연습문제 3: 사전 확률(Prior) 확인하기

학습된 `GaussianNB` 모델이 데이터로부터 학습한 각 클래스의 \*\*사전 확률(Prior Probability)\*\*을 확인해보세요. `gnb_pipeline`의 `classifier` 단계에 접근하여 `class_prior_` 속성을 출력하면 됩니다. 이 값은 전체 훈련 데이터의 클래스 비율과 유사할 것입니다.

In [ ]:
# 연습문제 3 코드
# 파이프라인에서 학습된 모델 접근
trained_gnb = gnb_pipeline.named_steps['classifier']

# 사전 확률 출력
# 여기에 코드를 작성하세요.
prior_probabilities = trained_gnb.class_prior_
print(f"클래스 0 (연체X)의 사전 확률: {prior_probabilities[0]:.4f}")
print(f"클래스 1 (연체O)의 사전 확률: {prior_probabilities[1]:.4f}")

# 훈련 데이터의 실제 클래스 비율과 비교
print("\n훈련 데이터의 실제 클래스 비율:")
print(y_train.value_counts(normalize=True))